# Saving and Loading Models

这个文档提供一种各种各样保存和加载PyTorch模型的方法。自由地阅读整份文档，或者只是跳到你需要的代码中。

当涉及到保存和加载模型，这里有三种核心功能你需要熟悉：
torch.nn.Module.load_state_dict:使用去序列化的state_dict来加载模型参数字典。对于state_dict上更多的信息，请看state_dict是什么这部分。
内容：
什么是state_dict？
推断中的保存和加载模型。
普通Checkpoint中的保存和加载模型。
在一个文件中保存多份模型。
从不同模型中热启动来使用参数。
横跨设备的保存和加载模型。

### What is a state_dict?
在PyTorch中，torch.nn.Module中可学习的参数(权重和偏差)在模型参数中(通过model.parameters()来启动)。state_dict是简单的python字典对象，它将每一层map到它的参数tensor。注意到只有对可学习参数的层(卷基层，线性层等等)和注册缓冲区(batchnorm’s running_mean)可以进入模型的state_dict。优化对象(torch.optim)同时有state_dict,其中包含优化器状态信息以及超参数使用。

因为state_dict 对象是python字典，它们可以容易的保存，更新，修改和储存，增加了许多模块化的pytorch模型和优化器。

### Example:
让我们使用分类教程中的教材从简单模型中研究state_dict。

In [ ]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

#### Output:

In [ ]:
Model's state_dict:
conv1.weight     torch.Size([6, 3, 5, 5])
conv1.bias   torch.Size([6])
conv2.weight     torch.Size([16, 6, 5, 5])
conv2.bias   torch.Size([16])
fc1.weight   torch.Size([120, 400])
fc1.bias     torch.Size([120])
fc2.weight   torch.Size([84, 120])
fc2.bias     torch.Size([84])
fc3.weight   torch.Size([10, 84])
fc3.bias     torch.Size([10])

Optimizer's state_dict:
state    {}
param_groups     [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [4675713712, 4675713784, 4675714000, 4675714072, 4675714216, 4675714288, 4675714432, 4675714504, 4675714648, 4675714720]}]

### Saving & Loading Model for Inference
### Save/Load state_dict (Recommended)
#### Save:

In [ ]:
torch.save(model.state_dict(), PATH)

#### Load:

In [ ]:
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

对于在推断中保存模型，我们只需要保存训练模型的学习参数，用torch.save()函数保存模型的state_dict，这会给你最有弹性的模型保存方式，这就是它为什么是推荐的模型保存方法。

一种共同的PyTorch传统是使用.pt或者.pth文件extension来保存模型。

记得你一定要在运行推断前使用model.eval()来设置dropout和batch normalization层去评估模式。如果你没有这么做的化就会产生不连续推断结果。

#### 注意到：
注意到load_state_dict()函数使用了字典对象，不是一个用来保存对象的路径。这意味着你必须在你通过它去加载load_state_dict()函数前去序列化那个保存好的state_dict。例如，你不能使用model.load_state_dict(PATH)来加载。

### Save/Load Entire Model
#### Save:

In [ ]:
torch.save(model, PATH)

#### Load:

In [ ]:
# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

这个save/load process过程使用最直觉语法和涉及最少数量的代码。这样保存模型会保存使用python pickle的整个模块。这种方法的劣势是因为序列化数据被捆绑在特定的类中，以及在模型被保存时使用确切的方向结构。这样做的原因是pickle不保存模型类本身。然而，它保存文件中包含类的路径，这是在加载时间时使用的。因为这样，你的代码可以通过很多种方式在其他项目或者refactors后分成一段一段的。

一个普通的pytorch惯例是去使用.pt/.pth文件extension来保存模型。

记得你必须call model.eval()在运行推断之前去设置dropout和batch normalization层来评估模式，如果这样失败的话会产生不连续的推断结果。

### Saving & Loading a General Checkpoint for Inference and/or Resuming Training
#### Save:


In [ ]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            ...
            }, PATH)

#### Load:

In [ ]:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

当保存一个普通的checkpoint时，使用inference或者resuming训练，你一定要保存比模型state_dict更多的东西。而且，你还要保存optimizer的state_dict,因为这个包含了可以更新模型训练的buffers和参数。还有就是你可能保存你训练过的epoch，最新的训练损失，外部的torch.nn.Embedding layers之类的。

为了保存多种零部件，把它们组装进字典或者使用torch.save()来序列化字典。一个普通的PyTorch惯例是使用.tar文件extension来保存这些checkpoints。

为了加载这些零部件，首先初始化模型和优化器，然后使用torch.load()加载局部字典。从这里，你可以通过查询字典来简单启动保存item。

记得你必须在运行推断前call model.eval()来设置dropout和batch normalization层来评估模式。如果不这样做，会产生不连续的推断结果。如果你希望来resuming训练，call model.train()来保证这些层在训练模式。

### Saving Multiple Models in One File
#### Save:


In [ ]:
torch.save({
            'modelA_state_dict': modelA.state_dict(),
            'modelB_state_dict': modelB.state_dict(),
            'optimizerA_state_dict': optimizerA.state_dict(),
            'optimizerB_state_dict': optimizerB.state_dict(),
            ...
            }, PATH)

#### Load:

In [ ]:
modelA = TheModelAClass(*args, **kwargs)
modelB = TheModelBClass(*args, **kwargs)
optimizerA = TheOptimizerAClass(*args, **kwargs)
optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()

在保存模型时，使用各种各样的torch.nn.Modules，例如GAN，sequence-to-sequence模型，或者组合模型，你可以按照同样的方法在你保存普通checkpoint的时候。换句话来说，保存每个模型的state_dict和对应的优化器的字典。正如上提到的，你可以保存人格其他items，那些可以帮助你resuming训练通过简单把他们贴到字典上。

为了加载模型，首先启动模型和优化器，然后使用torch.load()来加载局部字典。从那里，你通过简单地使用查询字典来使用保存好的items。

记得你必须在运行推断前call model.eval() 来设置dropout和batch normalization层来评估模式。如果没有这样做的话，会产生不连续的推断结果。如果你希望resuming训练，请call model.train()来设置这些层来训练模式。

### Warmstarting Model Using Parameters from a Different Model
#### Save:

In [ ]:
torch.save(modelA.state_dict(), PATH)

#### Load:

In [ ]:
modelB = TheModelBClass(*args, **kwargs)
modelB.load_state_dict(torch.load(PATH), strict=False)

局部加载模型或者加载一个局部模型是在迁移学习或者训练一个新的复杂模型中很常见的。调整训练参数，甚至仅仅是一小部分都是很有用的，会帮助热启动整个训练过程以及帮助你的模型聚合地比从零开始更快。

不管你是从state_dict一部分地加载模型，这样会错过一些关键的东西，或者加载一个比你加载的模型带有更多keys的state_dict
，你可以设置strict argument来否定load_state_dict()函数来忽略非匹配keys

如果你希望从一层到一层加载参数，然而部分的keys不匹配。简单匹配你加载的模型的keys与state_dict中的参数keys。

### Saving & Loading Model Across Devices 
### Save on GPU, Load on CPU

#### Save:

In [ ]:
torch.save(model.state_dict(), PATH)

#### Load:

In [ ]:
device = torch.device('cpu')
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))


当加载一个在GPU中训练的加载进CPU的模型，通过传给torch.device('cpu')给在torch.load()函数中的map_location argument。这样的话，在tensor中储存的内容会动态地重新使用map_location argument map到CPU设备中。

### Save on GPU, Load on GPU
#### Save:

In [ ]:
torch.save(model.state_dict(), PATH)

#### Load:

In [ ]:
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)
# Make sure to call input = input.to(device) on any input tensors that you feed to the model

当加载在GPU中训练和保存的模型时，简单地转化初始模型到CUDA优化模型使用model.to(torch.device('cuda'))。同时，确保使用.to(torch.device('cuda'))函数在全部模型输入中来准备数据。注意到calling my_tensor.to(device)返回一个GPU上的新copy的my_tensor。
my_tensor没有重写。因此，记得手动地重写tensor： my_tensor = my_tensor.to(torch.device('cuda'))。

### Save on CPU, Load on GPU
#### Save:

In [ ]:
torch.save(model.state_dict(), PATH)

#### Load:

In [ ]:
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))  # Choose whatever GPU device number you want
model.to(device)
# Make sure to call input = input.to(device) on any input tensors that you feed to the model

当在GPU上加载在CPU上训练的模型时，在torch.load()中设置map_location argument到cuda:device_id。这样加载模型到某个GPU设备。最后，确认来使用.to(torch.device('cuda'))函数在全部模型输入来准备数据对CUDA优化模型。注意到calling my_tensor.to(device)返回一个在GPU上的my_tensord的新copy。这样没有重写my_tensor。因此，记得手动地重写tensors:my_tensor = my_tensor.to(torch.device('cuda'))。


### Saving torch.nn.DataParallel Models
#### Save:

In [ ]:
torch.save(model.module.state_dict(), PATH)

#### Load:

In [ ]:
# Load to whatever device you want

torch.nn.DataParallel是一个模型包装器，它可以启动平行CPU设备。为了大致地保存DataParallel模型，使用model.module.state_dict()。这样，你拥有了随时随地加载任何设备的自由。

Total running time of the script: ( 0 minutes 0.000 seconds)